## img-text-img

此文件的作用是使用img2text，然后再拼接上自定义的风格词，，同时使用深度图来控制构图，然后再次出图片

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError
import math


## import json file

In [2]:
promptJson = 'api.json'

with open(promptJson) as f:
    prompt = json.load(f)

In [3]:
RunMode = {}
RunMode['local'] = '127.0.0.1:8188'
RunMode['localNetwork'] = '192.168.31.99:8188'
RunMode['pod'] = '2bzezpggl7sjvi-3000.proxy.runpod.net/'


# 设置运行模式，来实现在本机，runpod本地，runpod远程的切换
server_address = RunMode['localNetwork']
client_id = str(uuid.uuid4())

In [4]:
from io import BytesIO
def changeImageSize(width, height):




    # 最小的宽高为本机896，runpod1024




    limited = 896 if server_address == RunMode['pod'] else 1024




    minSize = min(width, height)





    # 缩放宽高，使得最窄的边等于limited




    if minSize > limited:




        scale = minSize / limited




        widthScale = int(width / scale)




        heightScale = int(height / scale)


    else:




        scale = limited / minSize




        widthScale = int(width * scale)




        heightScale = int(height * scale)





    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数




    widthScale = int(widthScale/64) * 64




    heightScale = int(heightScale/64) * 64





    return widthScale, heightScale





def checkImageExsit(imageDir, imageName):




    for file in os.listdir(imageDir):




        if file.startswith(imageName):




            return True




    return False






def queue_prompt(prompt):




    while True:




        try:




            p = {"prompt": prompt, "client_id": client_id}




            data = json.dumps(p).encode('utf-8')
            print(data)




            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)




            return json.loads(urllib.request.urlopen(req, timeout=5).read())




        except URLError as e:




            print("连接出错：\n", e.reason)





def get_history(prompt_id):




    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:




        return json.loads(response.read())





def get_image(filename, subfolder, folder_type):




    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}




    url_values = urllib.parse.urlencode(data)




    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:


        return response.read()





def handle_whitespace(string: str):




    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")





def parse_name(ckpt_name):




    path = ckpt_name




    filename = path.split("/")[-1]




    filename = filename.split(".")[:-1]




    filename = ".".join(filename)
    return filename






def calculate_sha256(file_path):




    sha256_hash = hashlib.sha256()





    with open(file_path, "rb") as f:




        # Read the file in chunks to avoid loading the entire file into memory




        for byte_block in iter(lambda: f.read(4096), b""):




            sha256_hash.update(byte_block)





    return sha256_hash.hexdigest()





def get_images(ws, prompt):




    prompt_id = queue_prompt(prompt)['prompt_id']




    output_images = {}




    while True:




        out = ws.recv()




        if isinstance(out, str):




            message = json.loads(out)




            if message['type'] == 'executing':




                data = message['data']




                if data['node'] is None and data['prompt_id'] == prompt_id:




                    break #Execution is done


        else:




            continue #previews are binary data





    history = get_history(prompt_id)[prompt_id]




    for o in history['outputs']:




        for node_id in history['outputs']:




            node_output = history['outputs'][node_id]




            if 'images' in node_output:




                images_output = []




                for image in node_output['images']:




                    image_data = get_image(image['filename'], image['subfolder'], image['type'])


                    images_output.append(image_data)




                output_images[node_id] = images_output

    return output_images
    




def saveImages(result, imageDir,imageName,comment,previewImage=0): 




    metadata = PngInfo()




    metadata.add_text("parameters", comment)  




    k = 0 


    for node_id in result:    




        if k < previewImage:




            k += 1




            continue    




        for image_data in result[node_id]:            




            image = Image.open(io.BytesIO(image_data))




            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"




            image.save(imagePath,pnginfo=metadata) 




            k += 1
    




def upload(host, image_url):

    import requests

    # 下载图片
    response = requests.get(image_url)
    image_data = response.content
    image_name = image_url.split("/")[-1]

    # 创建一个文件对象
    image_file = BytesIO(image_data)
    # 读取image_url最后面的文件名
    image_file.name = image_name

    # 创建post数据
    post = {
        "image": ('image.jpg', image_file),  # 这里的'image.jpg'是文件名，你可能需要根据实际情况修改
        "overwrite": "true",  # 可选，如果存在同名文件，将会被覆盖
        "type": "image",  # 可选，指定文件类型
        "subfolder": "uploads"  # 可选，指定子文件夹
    }

    # 发送POST请求
    upload_url = f"https://{host}/upload/image"
    print(upload_url)
    response = requests.post(upload_url, files=post)

    # 检查响应
    if response.status_code == 200:
        print("Upload successful!")
        print(response.text)
        return response.text
    else:
        print("Upload failed!")
        return "False"

In [5]:
# 子文件夹
subDir = "训练用"

# 设置图片保存路径
imageDir = f"f:/Poledriver/remake/{subDir}" if server_address == RunMode['localNetwork'] else f"../../../image-outputs/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

print(f'本次处理的文件夹是{subDir}，图片保存路径是{imageDir}')

本次处理的文件夹是训练用，图片保存路径是f:/Poledriver/remake/训练用


In [6]:

from PIL import Image
%pip install sd-parsers
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 4
originalImag = 'https://pics.dmm.co.jp/digital/video/1start021/1start021jp-19.jpg'
prompt_string = "(loli:1.5),(flat chest:1.5),big ass,"

image_url = upload(server_address,originalImag)
        

# 读取原图片名字，去掉后缀
originalImagName = originalImag.split("/")[-1].split(".")[0]
# 检测目标文件夹里是否已经存在该图片，如果存在，跳过
if checkImageExsit(imageDir, originalImagName):
    print(f"{originalImagName}已经存在，跳过")
    exit() 

megapixels = 1.2

seed = random.randint(0, 976242998978323)

# 下面是确定实际生成图片的大小，种子，以及保存图片信息
prompt['106']['inputs']['text'] = prompt_string
prompt['44']['inputs']['image'] = image_url
prompt['93']['inputs']['noise_seed']= seed        
prompt['95']['inputs']['megapixels'] = megapixels
prompt['123']['inputs']['amount'] = batch_size

steps = prompt['93']['inputs']['steps']
cfg = prompt['93']['inputs']['cfg']
sampler_name = prompt['93']['inputs']['sampler_name']
scheduler = prompt['93']['inputs']['scheduler']
positive = prompt['106']['inputs']['text']
negative = prompt['49']['inputs']['text']

modelname = prompt['4']['inputs']['ckpt_name']
basemodelname = parse_name(modelname)        
ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname if server_address != RunMode['pod'] else "../../../ComfyUI/models/checkpoints/" + modelname
modelhash = calculate_sha256(ckpt_path)[:10]

comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Model hash: {modelhash}, Model: {basemodelname}, Version: ComfyUI"    

print(f"图片的信息是：{comment}")  

# 连接api，生成并保存图片
start_time = time.time()    
images = get_images(ws, prompt) 
saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
end_time = time.time()
print(f"耗时{end_time-start_time}秒")



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
https://192.168.31.99:8188/upload/image


SSLError: HTTPSConnectionPool(host='192.168.31.99', port=8188): Max retries exceeded with url: /upload/image (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:997)')))